In [3]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "C:\Users\Mypc\AppData\Roaming\Python\Python311\site-packages\pip\__init__.py", line 14, in <module>
    from pip.utils import get_installed_distributions, get_prog
  File "C:\Users\Mypc\AppData\Roaming\Python\Python311\site-packages\pip\utils\__init__.py", line 27, in <module>
    from pip._vendor import pkg_resources
  File "C:\Users\Mypc\AppData\Roaming\Python\Python311\site-packages\pip\_vendor\pkg_resources\__init__.py", line 32, in <module>
    import symbol
ModuleNotFoundError: No module named 'symbol'


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import numpy as np, pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from matplotlib.pyplot import figure

ModuleNotFoundError: No module named 'keras'

In [ ]:
df= pd.read_csv('B+demand.csv')

In [ ]:
df.head()

In [ ]:
plt.plot(df['demand'][0:40])
plt.xlabel("Day")
plt.ylabel("Blood demand")

In [ ]:
Y_test=df['demand'][32:40]

In [ ]:
rolling = df['demand'][0:40].rolling(window=5)
ypred_MA = rolling.mean()
print(ypred_MA.head(10))

In [ ]:
figure(figsize=(20, 6), dpi=80)
x_range = range(1,41)
plt.plot(x_range,ypred_MA,marker = 'o',label="5-period MA")
plt.plot(x_range,df['demand'][0:40],label="demand",marker = 'o')
plt.xlim([1, 40])
plt.xlabel("Day")
plt.ylabel("Blood demand")
plt.legend()
plt.show()

In [ ]:
mse_MA=mean_squared_error(Y_test, ypred_MA[32:40])

In [ ]:
ypred_MA

In [ ]:
mse_MA

In [ ]:
from statsmodels.tsa.api import SimpleExpSmoothing

In [ ]:
error_es=np.zeros(9)
ax=np.zeros(9)
for x in range(1,10,1):
    ins = SimpleExpSmoothing(df['demand'][0:40]).fit(smoothing_level=x/10,optimized=False)
    ins_cast = ins.forecast(8).rename('alpha=x/10')
    pred_es=ins.fittedvalues
    ax[x-1]=x/10
    error_es[x-1]=mean_squared_error(Y_test, pred_es[32:40])    

In [ ]:
plt.plot(ax,error_es)
plt.xlabel("Alpha")
plt.ylabel("MSE")
plt.legend()

In [ ]:
ins = SimpleExpSmoothing(df['demand'][0:40]).fit(smoothing_level=0.1,optimized=False)
ins_cast = ins.forecast(8).rename('alpha=0.1')
ypred_es=pred_es=ins.fittedvalues
mse_es=mean_squared_error(Y_test, pred_es[32:40])

In [ ]:
mse_es

In [ ]:
figure(figsize=(20, 6), dpi=80)
x_range = range(1,41)
plt.plot(x_range,ypred_es,marker = 'o',label="Exponential Smoothing")
plt.plot(x_range,df['demand'][0:40],label="demand",marker = 'o')
plt.xlim([1, 40])
plt.xlabel("Day")
plt.ylabel("Blood demand")
plt.legend()
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
result = adfuller(df['demand'][0:40])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df['demand'].dropna(),lags=20,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df['demand'].dropna(),lags=20,ax=ax2)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model=ARIMA(df['demand'][0:40],order=(1,0,1))
model_fit=model.fit()
model_fit.summary()

In [ ]:
ypred=model_fit.predict(dynamic=False)

In [ ]:
ypred

In [ ]:
figure(figsize=(20, 6), dpi=80)
x_range = range(1,41)
plt.plot(x_range,ypred,marker = 'o',label="ARIMA forecasting")
plt.plot(x_range,df['demand'][0:40],label="demand",marker = 'o')
plt.xlim([1, 40])
plt.xlabel("Day")
plt.ylabel("Blood demand")
plt.legend()
plt.show()

In [ ]:
error_arima=mean_squared_error(Y_test, ypred[32:40])
error_arima

In [ ]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X),array(y)

In [ ]:
demandlist=df['demand'][0:40].tolist()

In [ ]:
demandarr=df['demand'][0:40].to_numpy()

In [ ]:
n_steps = 3
# split into samples
X, y = split_sequence(demandlist, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

In [ ]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
X_train=X[0:29]
X_test=X[29:37]
Y_train=y[0:29]
Y_test=y[29:37]

In [ ]:
X.shape

In [ ]:
model = Sequential()
model.add(LSTM(40, activation='relu',input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X_train, Y_train, epochs=700, verbose=0)

In [ ]:
Y_pred = model.predict(X_test, verbose=0)

In [ ]:
Y_pred

In [ ]:
figure(figsize=(8, 6), dpi=80)
x_range = range(1,9)
plt.plot(x_range,Y_pred,marker = 'o',label="LSTM forecasting")
plt.plot(x_range,Y_test,label="demand",marker = 'o')
plt.plot(x_range,ypred_MA[32:40],marker = 'o',label="3-period MA")
plt.plot(x_range,ypred_es[32:40],marker = 'o',label="Exponential Smoothing")
plt.plot(x_range,ypred[32:40],marker = 'o',label="ARIMA forecasting")
plt.xlim([1, 8])
plt.xlabel("Day")
plt.ylabel("Blood demand")
plt.legend()
plt.show()

In [ ]:
error=mean_squared_error(Y_test, Y_pred)
error